---
toc: false
comments: true
layout: post
title: Sprint 5 Review
description: Review of sprint 5 for review
permalink: /csp/sprint5/review
courses: { csp: {week: 18} }
---

## Intro requests for Review
- Purpose of Program: To allow users to share their cars and talk about their dream cars with similar interest individuals. Also to get help from knowledgable individuals.
- Purpose of Feature: To allow users to send their VIN to us to use on various other parts of the website.


## Input/Output requests (demo)
- Will do this live

## List Requests. Use of list, dictionaries, and database
- The .md file uses the API to get a response for each of the methods: post, get, put, delete. The post runs a refresh vehicles function with then uses the get to update the table. The get just updates the table. The put sends a request to the server and when the server responds back it updates the table. The delete removes a row from the database and table.
- The query in the API code looks for a SQLalchemy model
- in the _CRUD class I use the create, read, update, and delete methods to create lists in the database. The methods adds the data to the different dictionaries, like a certain make for the make column, which all add up to create a list for the user.


## Algorithmic Code Request

In [ ]:
# I didn't add the imports and other stuff, just the class

class VINDecodeAPI:
    class _CRUD(Resource):
        @token_required()
        def post(self):
            # Retrieve the current user from the context (via JWT or session)
            current_user = g.current_user  # This assumes you are using g.current_user, set in your token_required decorator

            # Check if the current user is authenticated
            if not current_user:
                return jsonify({"message": "User is not authenticated"}), 401

            # Parse the VIN from the request
            data = request.get_json()
            vin = data.get('vin')

            # Validate the VIN
            if not vin:
                return Response("{'message': 'VIN is required'}", status=400, mimetype='application/json')

            if len(vin) != 17:
                return Response("{'message': 'VIN must be 17 characters long'}", status=400, mimetype='application/json')

            # Request data from the NHTSA API
            response = requests.get(f"{NHTSA_API_URL}{vin}?format=json")
            
            if response.status_code != 200:
                return Response("{'message': 'Failed to retrieve data from NHTSA API'}", status=500, mimetype='application/json')

            vin_data = response.json().get('Results', [{}])[0]

            # Extract necessary details
            make = vin_data.get('Make')
            model = vin_data.get('Model')
            year = vin_data.get('ModelYear')
            engine_type = vin_data.get('FuelTypePrimary')

            if not all([make, model, year, engine_type]):
                return Response("{'message': 'Incomplete data from NHTSA API'}", status=500, mimetype='application/json')

            # Check if the vehicle already exists in the database
            existing_vehicle = Vehicle.query.filter_by(vin=vin).first()
            if existing_vehicle:
                return jsonify({"message": "Vehicle already exists in the database", "vehicle": existing_vehicle.read()})

            # Create a new Vehicle object, associating it with the current user
            new_vehicle = Vehicle(vin=vin, make=make, model=model, year=year, engine_type=engine_type, uid=current_user.id)

            try:
                # Save the vehicle to the database
                db.session.add(new_vehicle)
                db.session.commit()
            except SQLAlchemyError as e:
                db.session.rollback()
                return Response(f"{{'message': 'Database error. User could already have VIN: {str(e)}'}}", status=500, mimetype='application/json')

            return jsonify(new_vehicle.read())
        
        @token_required()
        def get(self):
            """
            Retrieve vehicles for the current user.

            Retrieves a list of vehicles associated with the authenticated user.

            Returns:
                JSON response with a list of vehicle dictionaries.
            """
            # Retrieve the current user from the token_required authentication check  
            current_user = g.current_user

            if not current_user:
                return jsonify({"message": "User is not authenticated"}), 401

            # Query the database for vehicles associated with the current user
            user_vehicles = Vehicle.query.filter_by(_uid=current_user.id).all()

            if not user_vehicles:
                return jsonify({"message": "No vehicles found for the current user"}), 404

            # Prepare a JSON list of the user's vehicles
            json_ready = [vehicle.read() for vehicle in user_vehicles]

            # Return response, a list of the user's vehicles in JSON format
            return jsonify(json_ready)

        @token_required()
        def put(self):
            # Retrieve the current user from the context
            current_user = g.current_user

            if not current_user:
                return jsonify({"message": "User is not authenticated"}), 401

            # Parse the data from the request
            data = request.get_json()
            old_vin = data.get('old_vin')  # The current VIN of the vehicle in the database
            new_vin = data.get('new_vin')  # The new VIN to update to

            # Validate the input
            if not old_vin or not new_vin:
                return Response("{'message': 'Both old_vin and new_vin are required'}", status=400, mimetype='application/json')

            if len(new_vin) != 17:
                return Response("{'message': 'new_vin must be 17 characters long'}", status=400, mimetype='application/json')

            # Find the vehicle by old VIN and ensure it belongs to the current user
            vehicle = Vehicle.query.filter_by(_vin=old_vin, _uid=current_user.id).first()
            if not vehicle:
                return Response("{'message': 'Vehicle not found or does not belong to the user'}", status=404, mimetype='application/json')

            # Fetch updated vehicle information from the NHTSA API
            response = requests.get(f"{NHTSA_API_URL}{new_vin}?format=json")

            if response.status_code != 200:
                return Response("{'message': 'Failed to retrieve data from NHTSA API'}", status=500, mimetype='application/json')

            vin_data = response.json().get('Results', [{}])[0]

            # Extract necessary details
            make = vin_data.get('Make')
            model = vin_data.get('Model')
            year = vin_data.get('ModelYear')
            engine_type = vin_data.get('FuelTypePrimary')

            if not all([make, model, year, engine_type]):
                return Response("{'message': 'Incomplete data from NHTSA API'}", status=500, mimetype='application/json')

            # Update the vehicle's VIN and other details
            try:
                vehicle.vin = new_vin  # Update VIN
                vehicle._make = make
                vehicle._model = model
                vehicle._year = year
                vehicle._engine_type = engine_type

                db.session.commit()
            except SQLAlchemyError as e:
                db.session.rollback()
                return Response(f"{{'message': 'Database error: {str(e)}'}}", status=500, mimetype='application/json')

            return jsonify({"message": "Vehicle VIN updated successfully", "vehicle": vehicle.read()})
        
        @token_required("Admin")
        def delete(self):
            """
            Delete a vehicle record by VIN.
            Only users with the 'Admin' role are authorized to delete vehicles.

            Returns:
                JSON response confirming deletion or an error message.
            """
            # Parse and normalize the VIN
            data = request.get_json()
            vin = data.get('vin', '').strip().upper()

            if not vin:
                return Response("{'message': 'VIN is required for deletion'}", status=400, mimetype='application/json')

            # Find the vehicle by VIN (stored as _vin in the database)
            from sqlalchemy import func
            vehicle = Vehicle.query.filter(func.upper(Vehicle._vin) == vin).first()

            if not vehicle:
                return Response(f"{{'message': 'Vehicle with VIN {vin} not found'}}", status=404, mimetype='application/json')

            try:
                # Delete the vehicle record
                db.session.delete(vehicle)
                db.session.commit()
                return jsonify({"message": f"Vehicle with VIN {vin} has been successfully deleted."})
            except SQLAlchemyError as e:
                db.session.rollback()
                return Response('')


- This class uses the flask_restful library to handle all of the _CRUD methods

- Parameters: body of request; { "vin": "12345678901234567" } This is an example where the post method accepts a json payload
- Return type: returns json via jsonify. Seen with response messages like {"message": "Vehicle not found"}

- The PUT method uses sequencing, selection and iteration. It uses sequencing because it flows from reading, to fetching to updating. It uses selection because it uses checks for user authentication, making sure inputs are correct (like not longer than 17 characters etc) and database queries. It also uses iteration by taking a response from the NHTSA API and extracting data.
- The put method is below:

In [ ]:
@token_required()
def put(self):
    current_user = g.current_user
    if not current_user:
        return jsonify({"message": "User is not authenticated"}), 401  # Selection

    data = request.get_json()
    old_vin = data.get('old_vin')
    new_vin = data.get('new_vin')
    if not old_vin or not new_vin or len(new_vin) != 17:
        return Response("{'message': 'Invalid input'}", status=400, mimetype='application/json')  # Selection

    vehicle = Vehicle.query.filter_by(_vin=old_vin, _uid=current_user.id).first()
    if not vehicle:
        return Response("{'message': 'Vehicle not found'}", status=404, mimetype='application/json')  # Selection

    response = requests.get(f"{NHTSA_API_URL}{new_vin}?format=json")
    vin_data = response.json().get('Results', [{}])[0]  # Iteration

    if not vin_data.get('Make'):
        return Response("{'message': 'Incomplete data from NHTSA API'}", status=500, mimetype='application/json')  # Selection

    vehicle.vin = new_vin
    db.session.commit()
    return jsonify({"message": "Vehicle VIN updated", "vehicle": vehicle.read()})


## Call to Algorithm Request
Example (of post) is below:

In [ ]:
const response = await fetch(`${pythonURI}/api/vinStore`, {
  method: "POST",
  headers: { 'Content-Type': 'application/json' },
  body: JSON.stringify({ vin: "12345678901234567" }),
});
const data = await response.json();


- The code above calls for the api and gets a response in json. This can then be used to display on the DOM
- return/response handling can be seen with successful or failed requests. With a success UI elements will be updates, while on a failure the elements will not be updated except for an error
- Changing data or method triggers different responses, as seen with a successful VIN post and an unsuccessful one. Submitting a unique and real VIN will be met with a success message, while a VIN that isn't real or isn't 17 characters long for instance will be met with an error
- Normal condition: {"message": "Vehicle added successfully", "vehicle": {"vin": "123...17", "make": "Toyota", "model": "Corolla"}}
- Error condition: {"message": "VIN must be 17 characters long"}